In [36]:
import xarray as xr

import climepi
import climepi.climdata as climdata
import climepi.epimod as epimod

In [38]:
ds_clim = climdata.get_example_dataset("isimip_london").sel(
    {"scenario": ["ssp126", "ssp370", "ssp585"]}
)
ds_clim

2024-02-07 18:41:13,603 [WARNING]: bounds.py(add_missing_bounds:186) >> Cannot generate bounds for coordinate variable 'lon' which has a length <= 1 (singleton).
2024-02-07 18:41:13,603 [WARNING]: bounds.py(add_missing_bounds:186) >> Cannot generate bounds for coordinate variable 'lon' which has a length <= 1 (singleton).
2024-02-07 18:41:13,609 [WARNING]: bounds.py(add_missing_bounds:186) >> Cannot generate bounds for coordinate variable 'lat' which has a length <= 1 (singleton).
2024-02-07 18:41:13,609 [WARNING]: bounds.py(add_missing_bounds:186) >> Cannot generate bounds for coordinate variable 'lat' which has a length <= 1 (singleton).


<xarray.Dataset>
Dimensions:        (scenario: 3, realization: 1, lat: 1, lon: 1, model: 10,
                    time: 1032, bnds: 2)
Coordinates:
  * scenario       (scenario) object 'ssp126' 'ssp370' 'ssp585'
  * realization    (realization) int64 0
  * lat            (lat) float64 51.75
  * lon            (lon) float64 -0.25
  * model          (model) object 'canesm5' 'cnrm-cm6-1' ... 'ukesm1-0-ll'
  * time           (time) object 2015-01-16 12:00:00 ... 2100-12-16 12:00:00
Dimensions without coordinates: bnds
Data variables:
    temperature    (scenario, model, realization, time, lat, lon) float64 dask.array<chunksize=(1, 1, 1, 1032, 1, 1), meta=np.ndarray>
    time_bnds      (time, bnds) object dask.array<chunksize=(1032, 2), meta=np.ndarray>
    precipitation  (scenario, model, realization, time, lat, lon) float64 dask.array<chunksize=(1, 1, 1, 1032, 1, 1), meta=np.ndarray>
Attributes:
    title:        ISIMIP3b bias-adjusted climate input data
    institution:  Potsdam Institute for Climate Impact Research (PIK)
    project:      Inter-Sectoral Impact Model Intercomparison Project phase 3...
    contact:      ISIMIP cross-sectoral science team <info@isimip.org> <https...
    summary:      CMIP6 daily output data bias-adjusted and statistically dow...
    references:   Lange (2019) <https://doi.org/10.5194/gmd-12-3055-2019> and...

In [39]:
ds_clim_ym = ds_clim.climepi.yearly_average()
ds_clim_ym.climepi.plot_time_series("temperature", by=["scenario"])
# ds_clim.isel(time=ds_clim.time.dt.year==2100).climepi.plot_time_series("temperature", color="r")

BokehModel(combine_events=True, render_bundle={'docs_json': {'40169bfc-96a5-4e29-8058-7e12a9e3b20f': {'version…

In [40]:
data_vars = ds_clim_ym.climepi.get_non_bnd_data_vars()
poly_coeff_data_vars = [data_var + "_polyfit_coefficients" for data_var in data_vars]
data_var_mapping = dict(zip(poly_coeff_data_vars, data_vars))
fitted_polys = ds_clim_ym[data_vars].polyfit(dim="time", deg=4, full=True)
ds_clim_ym_fit = xr.polyval(
    coord=ds_clim_ym.time,
    coeffs=fitted_polys[poly_coeff_data_vars],
).rename(data_var_mapping)

In [41]:
fitted_polys

<xarray.Dataset>
Dimensions:                             (degree: 5, scenario: 3, model: 10,
                                         realization: 1, lat: 1, lon: 1)
Coordinates:
  * degree                              (degree) int64 4 3 2 1 0
  * scenario                            (scenario) object 'ssp126' ... 'ssp585'
  * model                               (model) object 'canesm5' ... 'ukesm1-...
  * realization                         (realization) int64 0
  * lat                                 (lat) float64 51.75
  * lon                                 (lon) float64 -0.25
Data variables:
    time_matrix_rank                    int64 5
    time_singular_values                (degree) float64 2.157 ... 0.0004175
    temperature_polyfit_coefficients    (degree, scenario, model, realization, lat, lon) float64 dask.array<chunksize=(5, 1, 10, 1, 1, 1), meta=np.ndarray>
    temperature_polyfit_residuals       (scenario, model, realization, lat, lon) float64 dask.array<chunksize=(1, 10, 1, 1, 1), meta=np.ndarray>
    precipitation_polyfit_coefficients  (degree, scenario, model, realization, lat, lon) float64 dask.array<chunksize=(5, 1, 10, 1, 1, 1), meta=np.ndarray>
    precipitation_polyfit_residuals     (scenario, model, realization, lat, lon) float64 dask.array<chunksize=(1, 10, 1, 1, 1), meta=np.ndarray>
Attributes:
    title:        ISIMIP3b bias-adjusted climate input data
    institution:  Potsdam Institute for Climate Impact Research (PIK)
    project:      Inter-Sectoral Impact Model Intercomparison Project phase 3...
    contact:      ISIMIP cross-sectoral science team <info@isimip.org> <https...
    summary:      CMIP6 daily output data bias-adjusted and statistically dow...
    references:   Lange (2019) <https://doi.org/10.5194/gmd-12-3055-2019> and...

In [42]:
ds_clim_ym_fit

<xarray.Dataset>
Dimensions:        (scenario: 3, model: 10, realization: 1, lat: 1, lon: 1,
                    time: 86)
Coordinates:
  * scenario       (scenario) object 'ssp126' 'ssp370' 'ssp585'
  * model          (model) object 'canesm5' 'cnrm-cm6-1' ... 'ukesm1-0-ll'
  * realization    (realization) int64 0
  * lat            (lat) float64 51.75
  * lon            (lon) float64 -0.25
  * time           (time) object 2015-07-02 12:00:00 ... 2100-07-02 12:00:00
Data variables:
    temperature    (time, scenario, model, realization, lat, lon) float64 dask.array<chunksize=(86, 1, 10, 1, 1, 1), meta=np.ndarray>
    precipitation  (time, scenario, model, realization, lat, lon) float64 dask.array<chunksize=(86, 1, 10, 1, 1, 1), meta=np.ndarray>

In [43]:
p1 = ds_clim_ym_fit.squeeze().climepi.plot_time_series("temperature", by=["scenario"])
p2 = ds_clim_ym.squeeze().climepi.plot_time_series("temperature", by=["scenario"])
p1 * p2

BokehModel(combine_events=True, render_bundle={'docs_json': {'e5250e69-bfa1-48b4-a56e-fd8317078f23': {'version…

In [44]:
variance_internal = (ds_clim_ym - ds_clim_ym_fit).var()
variance_internal.load()

<xarray.Dataset>
Dimensions:        ()
Data variables:
    temperature    float64 0.3001
    precipitation  float64 0.07978

In [45]:
variance_model = ds_clim_ym_fit.var(dim="model").mean(dim="scenario")
variance_model.load()

<xarray.Dataset>
Dimensions:        (realization: 1, lat: 1, lon: 1, time: 86)
Coordinates:
  * realization    (realization) int64 0
  * lat            (lat) float64 51.75
  * lon            (lon) float64 -0.25
  * time           (time) object 2015-07-02 12:00:00 ... 2100-07-02 12:00:00
Data variables:
    temperature    (time, realization, lat, lon) float64 0.1954 0.1726 ... 1.737
    precipitation  (time, realization, lat, lon) float64 0.03245 ... 0.0368

In [46]:
variance_scenario = ds_clim_ym_fit.mean(dim="model").var(dim="scenario")
variance_scenario.load()

<xarray.Dataset>
Dimensions:        (realization: 1, lat: 1, lon: 1, time: 86)
Coordinates:
  * realization    (realization) int64 0
  * lat            (lat) float64 51.75
  * lon            (lon) float64 -0.25
  * time           (time) object 2015-07-02 12:00:00 ... 2100-07-02 12:00:00
Data variables:
    temperature    (time, realization, lat, lon) float64 0.005619 ... 2.806
    precipitation  (time, realization, lat, lon) float64 0.0003008 ... 0.001466

In [47]:
check = ds_clim_ym_fit.var(dim=["model", "scenario"]) - (
    variance_model + variance_scenario
)
check.load()

<xarray.Dataset>
Dimensions:        (realization: 1, lat: 1, lon: 1, time: 86)
Coordinates:
  * realization    (realization) int64 0
  * lat            (lat) float64 51.75
  * lon            (lon) float64 -0.25
  * time           (time) object 2015-07-02 12:00:00 ... 2100-07-02 12:00:00
Data variables:
    temperature    (time, realization, lat, lon) float64 0.0 ... -8.882e-16
    precipitation  (time, realization, lat, lon) float64 0.0 ... 6.939e-18